# Telecom Churn - Helping companies to identify  potential churn customers.

In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.<br/>
Main objectivies<br/>
1. Identify High profile/profitable customers who are planning to change network.
2. We perform analysis based on customer usage. 

In [1]:
# libraries
import pandas as pd
import numpy as np
from fancyimpute import KNN



# hide warnings
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'fancyimpute'

# EDA 
## Step 1 - Data Sourcing 
#### We load data collected to perform analysis

In [2]:
# dataset
telechurn = pd.read_csv("telecom_churn_data.csv")

In [3]:
telechurn.shape
data=telechurn

We have 99999 rows and 226 columns or features. Here label will be our Target and We analyze how this feature is dependent on remaining 225 features.

In [4]:
# What type of values are stored in the columns?
telechurn.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 226 columns):
mobile_number               int64
circle_id                   int64
loc_og_t2o_mou              float64
std_og_t2o_mou              float64
loc_ic_t2o_mou              float64
last_date_of_month_6        object
last_date_of_month_7        object
last_date_of_month_8        object
last_date_of_month_9        object
arpu_6                      float64
arpu_7                      float64
arpu_8                      float64
arpu_9                      float64
onnet_mou_6                 float64
onnet_mou_7                 float64
onnet_mou_8                 float64
onnet_mou_9                 float64
offnet_mou_6                float64
offnet_mou_7                float64
offnet_mou_8                float64
offnet_mou_9                float64
roam_ic_mou_6               float64
roam_ic_mou_7               float64
roam_ic_mou_8               float64
roam_ic_mou_9               float6

In [5]:
# Check the summary of the dataset
telechurn.describe(include='all')

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9,arpu_6,...,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g
count,9.999900e+04,99999.0,98981.0,98981.0,98981.0,99999,99398,98899,98340,99999.000000,...,99999.000000,25153.000000,25571.000000,26339.000000,25922.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000
unique,NaN,NaN,NaN,NaN,NaN,1,1,1,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,6/30/2014,7/31/2014,8/31/2014,9/30/2014,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,99999,99398,98899,98340,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,7.001207e+09,109.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,282.987358,...,0.084581,0.914404,0.908764,0.890808,0.860968,1219.854749,68.170248,66.839062,60.021204,3.299373
std,6.956694e+05,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,328.439770,...,0.650457,0.279772,0.287950,0.311885,0.345987,954.733842,267.580450,271.201856,253.938223,32.408353
min,7.000000e+09,109.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,-2258.709000,...,0.000000,0.000000,0.000000,0.000000,0.000000,180.000000,0.000000,0.000000,0.000000,0.000000
25%,7.000606e+09,109.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,93.411500,...,0.000000,1.000000,1.000000,1.000000,1.000000,467.000000,0.000000,0.000000,0.000000,0.000000
50%,7.001205e+09,109.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,197.704000,...,0.000000,1.000000,1.000000,1.000000,1.000000,863.000000,0.000000,0.000000,0.000000,0.000000
75%,7.001812e+09,109.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,371.060000,...,0.000000,1.000000,1.000000,1.000000,1.000000,1807.500000,0.000000,0.000000,0.000000,0.000000


In [6]:
telechurn.columns

Index(['mobile_number', 'circle_id', 'loc_og_t2o_mou', 'std_og_t2o_mou',
       'loc_ic_t2o_mou', 'last_date_of_month_6', 'last_date_of_month_7',
       'last_date_of_month_8', 'last_date_of_month_9', 'arpu_6',
       ...
       'sachet_3g_9', 'fb_user_6', 'fb_user_7', 'fb_user_8', 'fb_user_9',
       'aon', 'aug_vbc_3g', 'jul_vbc_3g', 'jun_vbc_3g', 'sep_vbc_3g'],
      dtype='object', length=226)

In [7]:
TypeCol =telechurn.columns.to_series().groupby(telechurn.dtypes).groups

In [8]:
TypeCol

{dtype('int64'): Index(['mobile_number', 'circle_id', 'total_rech_num_6', 'total_rech_num_7',
        'total_rech_num_8', 'total_rech_num_9', 'total_rech_amt_6',
        'total_rech_amt_7', 'total_rech_amt_8', 'total_rech_amt_9',
        'max_rech_amt_6', 'max_rech_amt_7', 'max_rech_amt_8', 'max_rech_amt_9',
        'last_day_rch_amt_6', 'last_day_rch_amt_7', 'last_day_rch_amt_8',
        'last_day_rch_amt_9', 'monthly_2g_6', 'monthly_2g_7', 'monthly_2g_8',
        'monthly_2g_9', 'sachet_2g_6', 'sachet_2g_7', 'sachet_2g_8',
        'sachet_2g_9', 'monthly_3g_6', 'monthly_3g_7', 'monthly_3g_8',
        'monthly_3g_9', 'sachet_3g_6', 'sachet_3g_7', 'sachet_3g_8',
        'sachet_3g_9', 'aon'],
       dtype='object'),
 dtype('float64'): Index(['loc_og_t2o_mou', 'std_og_t2o_mou', 'loc_ic_t2o_mou', 'arpu_6',
        'arpu_7', 'arpu_8', 'arpu_9', 'onnet_mou_6', 'onnet_mou_7',
        'onnet_mou_8',
        ...
        'night_pck_user_8', 'night_pck_user_9', 'fb_user_6', 'fb_user_7',
       

# Grouping Columns for Month 6

In [9]:
mnth_6_cols = telechurn.columns[telechurn.columns.str.endswith("_6")]
print(mnth_6_cols)

Index(['last_date_of_month_6', 'arpu_6', 'onnet_mou_6', 'offnet_mou_6',
       'roam_ic_mou_6', 'roam_og_mou_6', 'loc_og_t2t_mou_6',
       'loc_og_t2m_mou_6', 'loc_og_t2f_mou_6', 'loc_og_t2c_mou_6',
       'loc_og_mou_6', 'std_og_t2t_mou_6', 'std_og_t2m_mou_6',
       'std_og_t2f_mou_6', 'std_og_t2c_mou_6', 'std_og_mou_6', 'isd_og_mou_6',
       'spl_og_mou_6', 'og_others_6', 'total_og_mou_6', 'loc_ic_t2t_mou_6',
       'loc_ic_t2m_mou_6', 'loc_ic_t2f_mou_6', 'loc_ic_mou_6',
       'std_ic_t2t_mou_6', 'std_ic_t2m_mou_6', 'std_ic_t2f_mou_6',
       'std_ic_t2o_mou_6', 'std_ic_mou_6', 'total_ic_mou_6', 'spl_ic_mou_6',
       'isd_ic_mou_6', 'ic_others_6', 'total_rech_num_6', 'total_rech_amt_6',
       'max_rech_amt_6', 'date_of_last_rech_6', 'last_day_rch_amt_6',
       'date_of_last_rech_data_6', 'total_rech_data_6', 'max_rech_data_6',
       'count_rech_2g_6', 'count_rech_3g_6', 'av_rech_amt_data_6',
       'vol_2g_mb_6', 'vol_3g_mb_6', 'arpu_3g_6', 'arpu_2g_6',
       'night_pck_user

# Grouping Columns for Month 7

In [10]:
mnth_7_cols = telechurn.columns[telechurn.columns.str.endswith("_7")]
print(mnth_7_cols)

Index(['last_date_of_month_7', 'arpu_7', 'onnet_mou_7', 'offnet_mou_7',
       'roam_ic_mou_7', 'roam_og_mou_7', 'loc_og_t2t_mou_7',
       'loc_og_t2m_mou_7', 'loc_og_t2f_mou_7', 'loc_og_t2c_mou_7',
       'loc_og_mou_7', 'std_og_t2t_mou_7', 'std_og_t2m_mou_7',
       'std_og_t2f_mou_7', 'std_og_t2c_mou_7', 'std_og_mou_7', 'isd_og_mou_7',
       'spl_og_mou_7', 'og_others_7', 'total_og_mou_7', 'loc_ic_t2t_mou_7',
       'loc_ic_t2m_mou_7', 'loc_ic_t2f_mou_7', 'loc_ic_mou_7',
       'std_ic_t2t_mou_7', 'std_ic_t2m_mou_7', 'std_ic_t2f_mou_7',
       'std_ic_t2o_mou_7', 'std_ic_mou_7', 'total_ic_mou_7', 'spl_ic_mou_7',
       'isd_ic_mou_7', 'ic_others_7', 'total_rech_num_7', 'total_rech_amt_7',
       'max_rech_amt_7', 'date_of_last_rech_7', 'last_day_rch_amt_7',
       'date_of_last_rech_data_7', 'total_rech_data_7', 'max_rech_data_7',
       'count_rech_2g_7', 'count_rech_3g_7', 'av_rech_amt_data_7',
       'vol_2g_mb_7', 'vol_3g_mb_7', 'arpu_3g_7', 'arpu_2g_7',
       'night_pck_user

# Grouping Columns for Month 8

In [11]:
mnth_8_cols = telechurn.columns[telechurn.columns.str.endswith("_8")]
print(mnth_8_cols)

Index(['last_date_of_month_8', 'arpu_8', 'onnet_mou_8', 'offnet_mou_8',
       'roam_ic_mou_8', 'roam_og_mou_8', 'loc_og_t2t_mou_8',
       'loc_og_t2m_mou_8', 'loc_og_t2f_mou_8', 'loc_og_t2c_mou_8',
       'loc_og_mou_8', 'std_og_t2t_mou_8', 'std_og_t2m_mou_8',
       'std_og_t2f_mou_8', 'std_og_t2c_mou_8', 'std_og_mou_8', 'isd_og_mou_8',
       'spl_og_mou_8', 'og_others_8', 'total_og_mou_8', 'loc_ic_t2t_mou_8',
       'loc_ic_t2m_mou_8', 'loc_ic_t2f_mou_8', 'loc_ic_mou_8',
       'std_ic_t2t_mou_8', 'std_ic_t2m_mou_8', 'std_ic_t2f_mou_8',
       'std_ic_t2o_mou_8', 'std_ic_mou_8', 'total_ic_mou_8', 'spl_ic_mou_8',
       'isd_ic_mou_8', 'ic_others_8', 'total_rech_num_8', 'total_rech_amt_8',
       'max_rech_amt_8', 'date_of_last_rech_8', 'last_day_rch_amt_8',
       'date_of_last_rech_data_8', 'total_rech_data_8', 'max_rech_data_8',
       'count_rech_2g_8', 'count_rech_3g_8', 'av_rech_amt_data_8',
       'vol_2g_mb_8', 'vol_3g_mb_8', 'arpu_3g_8', 'arpu_2g_8',
       'night_pck_user

# Grouping Columns for Month 9

In [12]:
mnth_8_cols = telechurn.columns[telechurn.columns.str.endswith("_9")]
print(mnth_8_cols)

Index(['last_date_of_month_9', 'arpu_9', 'onnet_mou_9', 'offnet_mou_9',
       'roam_ic_mou_9', 'roam_og_mou_9', 'loc_og_t2t_mou_9',
       'loc_og_t2m_mou_9', 'loc_og_t2f_mou_9', 'loc_og_t2c_mou_9',
       'loc_og_mou_9', 'std_og_t2t_mou_9', 'std_og_t2m_mou_9',
       'std_og_t2f_mou_9', 'std_og_t2c_mou_9', 'std_og_mou_9', 'isd_og_mou_9',
       'spl_og_mou_9', 'og_others_9', 'total_og_mou_9', 'loc_ic_t2t_mou_9',
       'loc_ic_t2m_mou_9', 'loc_ic_t2f_mou_9', 'loc_ic_mou_9',
       'std_ic_t2t_mou_9', 'std_ic_t2m_mou_9', 'std_ic_t2f_mou_9',
       'std_ic_t2o_mou_9', 'std_ic_mou_9', 'total_ic_mou_9', 'spl_ic_mou_9',
       'isd_ic_mou_9', 'ic_others_9', 'total_rech_num_9', 'total_rech_amt_9',
       'max_rech_amt_9', 'date_of_last_rech_9', 'last_day_rch_amt_9',
       'date_of_last_rech_data_9', 'total_rech_data_9', 'max_rech_data_9',
       'count_rech_2g_9', 'count_rech_3g_9', 'av_rech_amt_data_9',
       'vol_2g_mb_9', 'vol_3g_mb_9', 'arpu_3g_9', 'arpu_2g_9',
       'night_pck_user

# Categorical Columns

In [13]:
cat_col = ['night_pck_user_6','night_pck_user_7','night_pck_user_8','night_pck_user_9','monthly_2g_6','monthly_2g_7','monthly_2g_8','monthly_2g_9','sachet_2g_6','sachet_2g_7','sachet_2g_8','sachet_2g_9','monthly_3g_6','monthly_3g_7','monthly_3g_8','monthly_3g_9','sachet_3g_6','sachet_3g_7','sachet_3g_8','sachet_3g_9','fb_user_6','fb_user_7','fb_user_8','fb_user_9']
print(cat_col)

['night_pck_user_6', 'night_pck_user_7', 'night_pck_user_8', 'night_pck_user_9', 'monthly_2g_6', 'monthly_2g_7', 'monthly_2g_8', 'monthly_2g_9', 'sachet_2g_6', 'sachet_2g_7', 'sachet_2g_8', 'sachet_2g_9', 'monthly_3g_6', 'monthly_3g_7', 'monthly_3g_8', 'monthly_3g_9', 'sachet_3g_6', 'sachet_3g_7', 'sachet_3g_8', 'sachet_3g_9', 'fb_user_6', 'fb_user_7', 'fb_user_8', 'fb_user_9']


# Step 2 - Data Cleaning

In [14]:
# To make sure that number of duplicate rows are zero
sum(telechurn.duplicated(subset = "mobile_number")) == 0

True

In [15]:
### Clean All data elements to remove leading or trailing white spaces
#telechurn.applymap(lambda x: x.strip() if type(x) is str else x)
telechurn.head(10)

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9,arpu_6,...,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g
0,7000842753,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,197.385,...,0,1.0,1.0,1.0,NaN,968,30.40,0.00,101.20,3.58
1,7001865778,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,34.047,...,0,NaN,1.0,1.0,NaN,1006,0.00,0.00,0.00,0.00
2,7001625959,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,167.690,...,0,NaN,NaN,NaN,1.0,1103,0.00,0.00,4.17,0.00
3,7001204172,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,221.338,...,0,NaN,NaN,NaN,NaN,2491,0.00,0.00,0.00,0.00
4,7000142493,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,261.636,...,0,0.0,NaN,NaN,NaN,1526,0.00,0.00,0.00,0.00
5,7000286308,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,50.258,...,0,NaN,NaN,NaN,NaN,1471,0.00,0.00,0.00,0.00
6,7001051193,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,429.023,...,0,NaN,NaN,NaN,NaN,1673,0.00,0.00,0.00,0.00
7,7000701601,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,1069.180,...,0,NaN,NaN,NaN,NaN,802,57.74,19.38,18.74,0.00
8,7001524846,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,378.721,...,0,NaN,1.0,1.0,NaN,315,21.03,910.65,122.16,0.00
9,7001864400,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,119.518,...,0,NaN,1.0,NaN,NaN,902,0.00,0.00,0.00,0.00


### Fill blank values to 0

In [16]:
# If customer had not recharged then we can consider it as 0.
telechurn['av_rech_amt_data_6'].fillna(0, inplace=True)
telechurn['av_rech_amt_data_7'].fillna(0, inplace=True)
telechurn['av_rech_amt_data_8'].fillna(0, inplace=True)
telechurn['av_rech_amt_data_9'].fillna(0, inplace=True)
telechurn['total_rech_data_6'].fillna(0, inplace=True)
telechurn['total_rech_data_7'].fillna(0, inplace=True)
telechurn['total_rech_data_8'].fillna(0, inplace=True)
telechurn['total_rech_data_9'].fillna(0, inplace=True)
telechurn['total_rech_amt_6'].fillna(0, inplace=True)
telechurn['total_rech_amt_7'].fillna(0, inplace=True)
telechurn['total_rech_amt_8'].fillna(0, inplace=True)
telechurn['total_rech_amt_9'].fillna(0, inplace=True)

In [17]:
# Since we have more columns looping to see null percentage is greater than 70% which indicates the columns to be dropped.

dropcolumns = {}
columns=[]
for col in telechurn.columns:
    if (telechurn[col].isnull().sum()*100/telechurn.shape[0]) > 70:
        dropcolumns[col] = telechurn[col].isnull().sum()*100/telechurn.shape[0]
        if (telechurn[col].dtype != 'O'): 
            columns.append(col)
print(dropcolumns)
print(columns)

{'date_of_last_rech_data_6': 74.84674846748467, 'date_of_last_rech_data_7': 74.42874428744287, 'date_of_last_rech_data_8': 73.66073660736608, 'date_of_last_rech_data_9': 74.07774077740777, 'max_rech_data_6': 74.84674846748467, 'max_rech_data_7': 74.42874428744287, 'max_rech_data_8': 73.66073660736608, 'max_rech_data_9': 74.07774077740777, 'count_rech_2g_6': 74.84674846748467, 'count_rech_2g_7': 74.42874428744287, 'count_rech_2g_8': 73.66073660736608, 'count_rech_2g_9': 74.07774077740777, 'count_rech_3g_6': 74.84674846748467, 'count_rech_3g_7': 74.42874428744287, 'count_rech_3g_8': 73.66073660736608, 'count_rech_3g_9': 74.07774077740777, 'arpu_3g_6': 74.84674846748467, 'arpu_3g_7': 74.42874428744287, 'arpu_3g_8': 73.66073660736608, 'arpu_3g_9': 74.07774077740777, 'arpu_2g_6': 74.84674846748467, 'arpu_2g_7': 74.42874428744287, 'arpu_2g_8': 73.66073660736608, 'arpu_2g_9': 74.07774077740777, 'night_pck_user_6': 74.84674846748467, 'night_pck_user_7': 74.42874428744287, 'night_pck_user_8': 7

In [18]:
# Check the summary of the dataset
telechurn[columns].describe(include='all')

,max_rech_data_6,max_rech_data_7,max_rech_data_8,max_rech_data_9,count_rech_2g_6,count_rech_2g_7,count_rech_2g_8,count_rech_2g_9,count_rech_3g_6,count_rech_3g_7,...,arpu_2g_8,arpu_2g_9,night_pck_user_6,night_pck_user_7,night_pck_user_8,night_pck_user_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9
count,25153.000000,25571.000000,26339.000000,25922.00000,25153.000000,25571.000000,26339.000000,25922.000000,25153.000000,25571.000000,...,26339.000000,25922.000000,25153.000000,25571.000000,26339.000000,25922.000000,25153.000000,25571.000000,26339.000000,25922.000000
mean,126.393392,126.729459,125.717301,124.94144,1.864668,2.044699,2.016288,1.781807,0.599133,0.621720,...,86.599478,93.712026,0.025086,0.023034,0.020844,0.015971,0.914404,0.908764,0.890808,0.860968
std,108.477235,109.765267,109.437851,111.36376,2.570254,2.768332,2.720132,2.214701,1.274428,1.394524,...,168.247852,171.384224,0.156391,0.150014,0.142863,0.125366,0.279772,0.287950,0.311885,0.345987
min,1.000000,1.000000,1.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-55.830000,-45.740000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25.000000,25.000000,25.000000,25.00000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
50%,145.000000,145.000000,145.000000,145.00000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,...,9.270000,14.800000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
75%,177.000000,177.000000,179.000000,179.00000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,...,122.070000,140.010000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,1555.000000,1555.000000,1555.000000,1555.00000,42.000000,48.000000,44.000000,40.000000,29.000000,35.000000,...,3483.170000,3467.170000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [19]:
## Dropping columns for Date last rech data 

telechurn.drop(['date_of_last_rech_data_6','date_of_last_rech_data_7','date_of_last_rech_data_8', 'date_of_last_rech_data_9'], axis=1, inplace=True)

## If Maxiumn Recharge for Data is null we can consider it as 0 assuming no data recharge was done for the month.

telechurn['max_rech_data_6'].fillna(0, inplace=True)
telechurn['max_rech_data_7'].fillna(0, inplace=True)
telechurn['max_rech_data_8'].fillna(0, inplace=True)
telechurn['max_rech_data_9'].fillna(0, inplace=True)

## Customers who are not using 2g or 3g then they dont recharge for that service.
## If their was no recharge made then we will consider it as 0.
telechurn['count_rech_2g_6'].fillna(0, inplace=True)
telechurn['count_rech_2g_7'].fillna(0, inplace=True)
telechurn['count_rech_2g_8'].fillna(0, inplace=True)
telechurn['count_rech_2g_9'].fillna(0, inplace=True)

telechurn['count_rech_3g_6'].fillna(0, inplace=True)
telechurn['count_rech_3g_7'].fillna(0, inplace=True)
telechurn['count_rech_3g_8'].fillna(0, inplace=True)
telechurn['count_rech_3g_9'].fillna(0, inplace=True)

## Average revenue per user is total revenue for that service/ total users using the service. If ARPU is blank
## Then user is not using the service so their is no loss or gain hence we can consider as 0

telechurn['arpu_3g_6'].fillna(0, inplace=True)
telechurn['arpu_3g_7'].fillna(0, inplace=True)
telechurn['arpu_3g_8'].fillna(0, inplace=True)
telechurn['arpu_3g_9'].fillna(0, inplace=True)

telechurn['arpu_2g_6'].fillna(0, inplace=True)
telechurn['arpu_2g_7'].fillna(0, inplace=True)
telechurn['arpu_2g_8'].fillna(0, inplace=True)
telechurn['arpu_2g_9'].fillna(0, inplace=True)



# Categorical Columns

In [20]:
## We have more that 70 % as null for Social Networking Service (FaceBook..) 
## This is whether user is using social networking services or not.
## For null we can consider that user is not using social networking hence setting it to 0.
telechurn['fb_user_6'].fillna(0, inplace=True)
telechurn['fb_user_7'].fillna(0, inplace=True)
telechurn['fb_user_8'].fillna(0, inplace=True)
telechurn['fb_user_9'].fillna(0, inplace=True)

## Night pack if user is not using then lets set to 0 

telechurn['night_pck_user_6'].fillna(0, inplace=True)
telechurn['night_pck_user_7'].fillna(0, inplace=True)
telechurn['night_pck_user_8'].fillna(0, inplace=True)
telechurn['night_pck_user_9'].fillna(0, inplace=True)

In [21]:
# Since we have more columns looping to see null percentage is greater than 5% which indicates the columns to be dropped.

dropcolumns = {}
for col in telechurn.columns:
    if (telechurn[col].isnull().sum()*100/telechurn.shape[0]) > 5:
        dropcolumns[col] = telechurn[col].isnull().sum()*100/telechurn.shape[0]
print(dropcolumns)

{'onnet_mou_8': 5.3780537805378055, 'onnet_mou_9': 7.745077450774508, 'offnet_mou_8': 5.3780537805378055, 'offnet_mou_9': 7.745077450774508, 'roam_ic_mou_8': 5.3780537805378055, 'roam_ic_mou_9': 7.745077450774508, 'roam_og_mou_8': 5.3780537805378055, 'roam_og_mou_9': 7.745077450774508, 'loc_og_t2t_mou_8': 5.3780537805378055, 'loc_og_t2t_mou_9': 7.745077450774508, 'loc_og_t2m_mou_8': 5.3780537805378055, 'loc_og_t2m_mou_9': 7.745077450774508, 'loc_og_t2f_mou_8': 5.3780537805378055, 'loc_og_t2f_mou_9': 7.745077450774508, 'loc_og_t2c_mou_8': 5.3780537805378055, 'loc_og_t2c_mou_9': 7.745077450774508, 'loc_og_mou_8': 5.3780537805378055, 'loc_og_mou_9': 7.745077450774508, 'std_og_t2t_mou_8': 5.3780537805378055, 'std_og_t2t_mou_9': 7.745077450774508, 'std_og_t2m_mou_8': 5.3780537805378055, 'std_og_t2m_mou_9': 7.745077450774508, 'std_og_t2f_mou_8': 5.3780537805378055, 'std_og_t2f_mou_9': 7.745077450774508, 'std_og_t2c_mou_8': 5.3780537805378055, 'std_og_t2c_mou_9': 7.745077450774508, 'std_og_mo

### After closely reviewing above anything with null incomming or outgoing is indication that their was no incoming calls or outgoing calls whether is is location, T 2 T operator, ISD, STD or others.<Br/>
### Hence setting it to 0

In [22]:
for col in dropcolumns:
    telechurn[col].fillna(0, inplace=True)

### Setting last date of Month, which we know the data.

In [23]:
telechurn['last_date_of_month_7'].fillna('7/31/2014', inplace=True)
telechurn['last_date_of_month_8'].fillna('8/31/2014', inplace=True)
telechurn['last_date_of_month_9'].fillna('9/30/2014', inplace=True)
telechurn['last_date_of_month_9'].unique()

array(['9/30/2014'], dtype=object)

In [24]:
# Since we have more columns looping to see null percentage is greater than 0% which indicates the columns to be dropped.

dropcolumns = {}
for col in telechurn.columns:
    if (telechurn[col].isnull().sum()*100/telechurn.shape[0]) > 0:
        dropcolumns[col] = telechurn[col].isnull().sum()*100/telechurn.shape[0]
print(dropcolumns)

{'loc_og_t2o_mou': 1.018010180101801, 'std_og_t2o_mou': 1.018010180101801, 'loc_ic_t2o_mou': 1.018010180101801, 'onnet_mou_6': 3.937039370393704, 'onnet_mou_7': 3.8590385903859037, 'offnet_mou_6': 3.937039370393704, 'offnet_mou_7': 3.8590385903859037, 'roam_ic_mou_6': 3.937039370393704, 'roam_ic_mou_7': 3.8590385903859037, 'roam_og_mou_6': 3.937039370393704, 'roam_og_mou_7': 3.8590385903859037, 'loc_og_t2t_mou_6': 3.937039370393704, 'loc_og_t2t_mou_7': 3.8590385903859037, 'loc_og_t2m_mou_6': 3.937039370393704, 'loc_og_t2m_mou_7': 3.8590385903859037, 'loc_og_t2f_mou_6': 3.937039370393704, 'loc_og_t2f_mou_7': 3.8590385903859037, 'loc_og_t2c_mou_6': 3.937039370393704, 'loc_og_t2c_mou_7': 3.8590385903859037, 'loc_og_mou_6': 3.937039370393704, 'loc_og_mou_7': 3.8590385903859037, 'std_og_t2t_mou_6': 3.937039370393704, 'std_og_t2t_mou_7': 3.8590385903859037, 'std_og_t2m_mou_6': 3.937039370393704, 'std_og_t2m_mou_7': 3.8590385903859037, 'std_og_t2f_mou_6': 3.937039370393704, 'std_og_t2f_mou_7'

### All Columns except 'date_of_last_rech_6','date_of_last_rech_7', 'date_of_last_rech_8', 'date_of_last_rech_9' are either incoming or outgoing minutes of usage. Hence we can set to 0

In [25]:
## Columns not to drop
lstNoDrp = ['date_of_last_rech_6','date_of_last_rech_7', 'date_of_last_rech_8', 'date_of_last_rech_9']

for col in dropcolumns:
    if col not in lstNoDrp:
        telechurn[col].fillna(0, inplace=True)

In [26]:
# Since we have more columns looping to see null percentage is greater than 0% which indicates the columns to be dropped.

dropcolumns = {}
for col in telechurn.columns:
    if (telechurn[col].isnull().sum()*100/telechurn.shape[0]) > 0:
        dropcolumns[col] = telechurn[col].isnull().sum()*100/telechurn.shape[0]
print(dropcolumns)

{'date_of_last_rech_6': 1.6070160701607017, 'date_of_last_rech_7': 1.7670176701767017, 'date_of_last_rech_8': 3.6220362203622036, 'date_of_last_rech_9': 4.760047600476005}


# The only way to address above column is to store as day rather entire date. We know the month hence 0 means customer is inactive, 1 means last recharge was on 1st of the month. 

This information might be useful since customers might recharge during their paycheck and can be high value customer. 

# Step 3 - Derive New Attributes 

Lets Derive Day fields for above columns so that we can drop those.
Note 
0 - Customer did not recharge
1 - Customer last recharged was on 1st day of the month
10 - Customer last recharged was on 10th day of the month.

In [ ]:
telechurn['day_of_last_rech_6'] = telechurn['date_of_last_rech_6'].apply(lambda x: 0 if pd.isnull(x) else pd.to_datetime(x).day)
telechurn['day_of_last_rech_7'] = telechurn['date_of_last_rech_7'].apply(lambda x: 0 if pd.isnull(x) else pd.to_datetime(x).day)
telechurn['day_of_last_rech_8'] = telechurn['date_of_last_rech_8'].apply(lambda x: 0 if pd.isnull(x) else pd.to_datetime(x).day)
telechurn['day_of_last_rech_9'] = telechurn['date_of_last_rech_9'].apply(lambda x: 0 if pd.isnull(x) else pd.to_datetime(x).day)

In [ ]:
print(telechurn['day_of_last_rech_6'].unique())
print(telechurn['day_of_last_rech_7'].unique())
print(telechurn['day_of_last_rech_8'].unique())
print(telechurn['day_of_last_rech_9'].unique())

Since we derived new attributes for date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8 and date_of_last_rech_9, now lets drop the columns 

In [ ]:
telechurn.drop(['date_of_last_rech_6','date_of_last_rech_7','date_of_last_rech_8','date_of_last_rech_9'], axis=1, inplace=True)

Check for missing values

In [ ]:
# Since we have more columns looping to see null percentage is greater than 0% which indicates the columns to be dropped.

dropcolumns = {}
for col in telechurn.columns:
    if (telechurn[col].isnull().sum()*100/telechurn.shape[0]) > 0:
        dropcolumns[col] = telechurn[col].isnull().sum()*100/telechurn.shape[0]
print(dropcolumns)

All Missing values are fixed or filled.

Calculating Total Reach Amount for Data and finding overall amount reacharged by customer

In [ ]:
## Total recharge data amount
telechurn['total_data_rech_amt_6'] = telechurn['total_rech_data_6'] * telechurn['av_rech_amt_data_6'] 
telechurn['total_data_rech_amt_7'] = telechurn['total_rech_data_7'] * telechurn['av_rech_amt_data_7'] 
telechurn['total_data_rech_amt_8'] = telechurn['total_rech_data_8'] * telechurn['av_rech_amt_data_8'] 
telechurn['total_data_rech_amt_9'] = telechurn['total_rech_data_9'] * telechurn['av_rech_amt_data_9'] 

## Final overall recharge amount
telechurn['amt_rech_6'] = telechurn['total_data_rech_amt_6'] + telechurn['total_rech_amt_6']
telechurn['amt_rech_7'] = telechurn['total_data_rech_amt_7'] + telechurn['total_rech_amt_7']
telechurn['amt_rech_8'] = telechurn['total_data_rech_amt_8'] + telechurn['total_rech_amt_8']
telechurn['amt_rech_9'] = telechurn['total_data_rech_amt_9'] + telechurn['total_rech_amt_9']

Identifying the 70th Percentile of average amount.

In [ ]:
## Average amount for 6th and 7th Month
telechurn['average_recharge_amount_6_7'] = telechurn[['amt_rech_6','amt_rech_7']].mean(axis=1)
telechurn[['average_recharge_amount_6_7','amt_rech_6', 'amt_rech_7']].head()

In [ ]:
## Identifying 70th Percentile for Average Recharge amount
amt_70_percentile = np.percentile(telechurn['average_recharge_amount_6_7'], 70.0)
## Alternative Way to get percentile
## amt_70_percentile = telechurn['average_recharge_amount_6_7'].quantile(0.7)
print(amt_70_percentile)

# High Value Customers
Those who have recharged with an amount more than or equal to X, where X is the 70th percentile of the average recharge amount in the first two months (the good phase).

In [ ]:
hvc = telechurn.loc[(telechurn.average_recharge_amount_6_7 >= amt_70_percentile)]

In [ ]:
hvc.shape

We have total of 30001 High Value customers

In [ ]:
## Data Check for null Values, based on result Churn columns has no null values.
hvc[['total_ic_mou_9','total_og_mou_9','vol_2g_mb_9','vol_3g_mb_9']].isnull().sum()*100/hvc.shape[0]

In [ ]:
## Method to identify whether customer is churn or not based on October data.
def churn_cust(df):
    if ((df.total_ic_mou_9 == 0) and (df.total_og_mou_9 == 0) and (df.vol_2g_mb_9 == 0)and (df.vol_3g_mb_9 == 0)):
        return 1
    else: return 0

In [ ]:
## Calculate Churn information
hvc['churn'] = hvc.apply(churn_cust, axis = 1)

In [ ]:
## Validate churn information.
hvc[['total_ic_mou_9','total_og_mou_9','vol_2g_mb_9','vol_3g_mb_9','churn']].head()

# Data Cleaning or Fill Missing Values

In [ ]:
# Since we have more columns looping to see null percentage is greater than 35% which indicates the columns to be dropped.
hvc_1 = hvc
dropcolumns = {}
for col in hvc_1.columns:
    if (hvc_1[col].isnull().sum()*100/hvc_1.shape[0]) > 0:
        dropcolumns[col] = round(hvc_1[col].isnull().sum()*100/hvc_1.shape[0],2)
print(dropcolumns)